### The logic of the notebook

In order to be able to perform multiple operations I have designed the system as follows:

- first, I create the most basic function which just updates the owner of a player
- then I defined the class transfer, with argument name of the player
    - the class transfer can perform all market operations and then save database entries
    - however, it is not convenient to save directly to the database, in case there are accidental mistakes
- so I created another class called market session:
    - first, you start a market session and create each transfer within the session using the inner class transfer
    - when you are done with all the transfers, and are sure there are no mistakes you confirm the session
    - if you notice a mistake you can cancel the session and the databases rest unchanged
    
    
## OPEN QUESTIONS:

- what is the cost for draft? I set it to 0 for now
- is this the best way to define exchanges?
- we could in principle include in the player's info last transfer operation's id, but it is enough to look at the most recent in the transfer db

In [1]:
import pandas as pd
import numpy as np
import datetime
from datetime import date
import json
import copy
from pymongo import MongoClient
from pprint import pprint
import pymongo

from datetime import date

with open('credential.json','r') as f:
    cred = json.load(f)

cluster = MongoClient(cred['cred'])
# choosing database
db = cluster["Game"]
# choosing collection
collection = db["Players"]

/Users/lucaagozzino/opt/anaconda3/lib/python3.7/site-packages/pymongo/common.py:559: UserWarning: Unknown option retryWrites
  warnings.warn(str(exc))


In [2]:
collection.find_one({'name': 'MUSSO'})

{'_id': 2792,
 'name': 'MUSSO',
 'info': {'personal_info': {'full_name': 'Musso Juan',
   'birthdate': '06/05/1994',
   'nation': 'Argentina, Italia',
   'team_real': 'Udinese',
   'FC_role': 'P'},
  'contract': {'owner': 'pietro',
   'start_date': '01/08/2020',
   'cost': 0,
   'acquisition_mode': '',
   'previous_owner': '',
   'quotation_initial': 11},
  'current_team': {'owner': 'pietro',
   'squad': 'main',
   'start_date': '01/08/2020',
   'previous_team': '',
   'quotation_initial': 11,
   'on_loan': False},
  'stats': {'Pg': 4,
   'Mv': 5.88,
   'Mf': 4.38,
   'Gf': 0,
   'Gs': 6,
   'Rp': 0,
   'Rc': 0,
   'R+': 0,
   'R-': 0,
   'Ass': 0,
   'Asf': 0,
   'Amm': 0,
   'Esp': 0,
   'Au': 0}}}

In [3]:
"""
Function documentation:
This is the general function to update the owner of a player.
Arguments:

- new_owner: string, the new owner of the team (pietro, luca,...)
- mode: string, the acquisition mode (Scambio, draft,...)
- squad: assignment in the new team (main or primavera)
- name/id: string, use one OR the other as a filter to query the database
- time_stamp: boolean. True if you want to register the date of operation in the database
- rest: details to access database, set to default values

Note: I'm planning to use this function inside higher level functions, which is why I have the timestamp option, 
if you are doing tests there is no reason to update the date, but if you include this in a market operation function
(for instance exchange players) you want to register this and the higher level function will also create the database entry

"""

def update_owner(new_owner, mode, squad, loan = False, move_up = False, name = False, Id = False, time_stamp = False, CR = cred['cred'], DB = 'Game', CO = 'Players'):
    
    cluster = MongoClient(CR)
    db = cluster[DB]
    collection = db[CO]
    
    today = date.today()
    d1 = today.strftime("%Y/%m/%d")
    
    if name:
        filt = {'name': name}
    elif Id:
        filt = {'_id': str(Id)}
    else:
        print('Please enter name or Id argument')
        
        return None
        
    dic = collection.find_one(filt)
    old_owner = dic['info']['contract']['owner'] #when database structure changes, Modify this and following accordingly
    old_team = dic['info']['current_team']['owner'] +', ' + dic['info']['current_team']['squad']
    
    if loan:

        collection.update_one(filt, {'$set':{'info.current_team.previous_team': old_team}}) 
        collection.update_one(filt, {'$set':{'info.current_team.owner': new_owner}})
        collection.update_one(filt, {'$set':{'info.current_team.squad': squad}})
        
        
        if time_stamp:
            collection.update_one(filt, {'$set':{'info.current_team.start_date': d1}}) #
            print("Operation date: ")
            return d1
    
    elif move_up:
        
        collection.update_one(filt, {'$set':{'info.current_team.previous_team': old_team}}) 
        collection.update_one(filt, {'$set':{'info.current_team.squad': squad}})
        
        if time_stamp:
            collection.update_one(filt, {'$set':{'info.current_team.start_date': d1}}) #
            print("Operation date: ")
            return d1
        
        
    else:
    
        collection.update_one(filt, {'$set':{'info.contract.acquisition_mode': mode}})
        collection.update_one(filt, {'$set':{'info.contract.previous_owner': old_owner}})
        collection.update_one(filt, {'$set':{'info.contract.owner': new_owner}})

        collection.update_one(filt, {'$set':{'info.current_team.previous_team': old_team}}) 
        collection.update_one(filt, {'$set':{'info.current_team.owner': new_owner}})
        collection.update_one(filt, {'$set':{'info.current_team.squad': squad}})
    
        if time_stamp:
            collection.update_one(filt, {'$set':{'info.contract.start_date': d1}})
            collection.update_one(filt, {'$set':{'info.current_team.start_date': d1}}) #redundant
            print("Operation date: ")
            return d1

In [4]:
def update_Tr_DB(transfer_data, CR = cred['cred'], DB = 'Game', COT = 'Transfers'):
    cluster = MongoClient(CR)
    db = cluster[DB]
    collection = db[COT]
    
    now = datetime.datetime.today()
    Id = now.strftime("%Y%m%d%H%M%S")
    
    dic0 = transfer_data[0]
    
    if dic0['operation'] in ['Asta', 'Draft', 'Acquisto', 'Prestito', 'Fine Prestito', 'Promozione']:
        dic0['_id'] = Id
        
        collection.insert_one(dic0)
    
    if dic0['operation'] == 'Scambio':
        
        dic1 = transfer_data[1]
        
        
        dic0['_id'] = Id
        dic1['_id'] = str(int(Id) + 1)
        
        collection.insert_one(dic0)
        collection.insert_one(dic1)
    

In [5]:
"""
This is an inner level class, it performs all the market operations and creates database entries
But we'll use it inside another class, so all operations need to be confirmed before being saved

"""


class transfer:
    
    #need to add quotation info everywhere, maybe as return of transf_db

    def __init__(self, name, CR, DB, CO, COT):
        
        cluster = MongoClient(CR)
        db = cluster[DB]
        
        self.collection = db[CO]
        self.name = name
        self.info = collection.find_one({'name': name})
        self.CC = (CR, DB, CO)
        self.COT = COT
        
    def draft_in(self, new_owner, squad, cost = 0):
        (CR, DB, CO) = self.CC
        #print(new_owner+' has acquired '+self.name+' through draft.\nAssigned to '+squad+' squad\n')
        d1 = update_owner(new_owner = new_owner, mode = 'Draft', squad = squad, name = self.name, time_stamp = True, CR = CR, DB = DB, CO = CO)
        
        self.collection.update_one({'name': self.name},{'$set':{'info.contract.cost': cost}})
        
        
        transfer_data = [{
            'name': self.name,
            'operation': 'Draft',
            'new_owner': new_owner,
            'squad': squad,
            'cost': cost,
            'date': d1
                      }]
        
        
        update_Tr_DB(transfer_data, CR, DB, self.COT)
        #transf_db_update() #function to be defined
        
        
        
        return d1
    
    def auction_in(self, new_owner, squad, cost):
        (CR, DB, CO) = self.CC
        #print(new_owner+' has acquired '+self.name+' through auction for '+str(cost)+' FM.\nAssigned to '+squad+' squad\n')
        d1 = update_owner(new_owner = new_owner, mode = 'Asta', squad = squad, name = self.name, time_stamp = True, CR = CR, DB = DB, CO = CO)
        
        self.collection.update_one({'name': self.name},{'$set':{'info.contract.cost': cost}})
        #transf_db_update() #function to be defined, includes budgets adjustments
        
        transfer_data = [{
            'name': self.name,
            'operation': 'Asta',
            'new_owner': new_owner,
            'squad': squad,
            'cost': cost,
            'date': d1
                      }]
        
        
        update_Tr_DB(transfer_data, CR, DB, self.COT)
        
        
        
        return d1
    
    def cash_in(self, new_owner, squad, cost):
        old_owner = self.info['info']['contract']['owner']
        (CR, DB, CO) = self.CC
        #print(new_owner+' has acquired '+self.name+' through auction for '+str(cost)+' FM.\nAssigned to '+squad+' squad\n')
        d1 = update_owner(new_owner = new_owner, mode = 'Acquisto', squad = squad, name = self.name, time_stamp = True, CR = CR, DB = DB, CO = CO)
        
        self.collection.update_one({'name': self.name},{'$set':{'info.contract.cost': cost}})
        #transf_db_update() #function to be defined, includes budgets adjustments
        
        transfer_data = [{
            'name': self.name,
            'operation': 'Acquisto',
            'new_owner': new_owner,
            'old_owner': old_owner,
            'squad': squad,
            'cost': cost,
            'date': d1
                      }]
        
        
        update_Tr_DB(transfer_data, CR, DB, self.COT)
        
        return d1
    
    def exchange_with(self, exch_player, cost, squad, squad_exch_pl):
        
        #THIS IS PROBABLY THE MOST PROBLEMATIC, MAYBE WE NEED A BETTER WAY TO DEFINE IT
        #MAYBE JUST RENAMING SOME VARIABLES WILL SUFFICE
        
        (CR, DB, CO) = self.CC
        collection = self.collection
        self.exch_player = exch_player
        
        
        exch_info = collection.find_one({'name': exch_player})
        exch_owner = exch_info['info']['contract']['owner']
        owner = self.info['info']['contract']['owner']
        
        
        d1 = update_owner(new_owner = exch_owner, mode = 'Scambio', squad = squad, name = self.name, time_stamp = False, CR = CR, DB = DB, CO = CO)
        d1 = update_owner(new_owner = owner, mode = 'Scambio', squad = squad_exch_pl, name = exch_player, time_stamp = True, CR = CR, DB = DB, CO = CO)
        
        self.collection.update_one({'name': self.name},{'$set':{'info.contract.cost': max(0, cost)}})
        self.collection.update_one({'name': exch_player},{'$set':{'info.contract.cost': max(0, - cost)}})
        
        
        transfer_data_0 = {
            'name': self.name,
            'operation': 'Scambio',
            'new_owner': exch_owner,
            'exchange_player': exch_player,
            'squad': squad,
            'cost': max(0, cost),
            'date': d1
                      }
        transfer_data_1 = {
            'name': exch_player,
            'operation': 'Scambio',
            'new_owner': owner,
            'exchange_player': self.name,
            'squad': squad_exch_pl,
            'cost': max(0, -cost),
            'date': d1
                      }
        transfer_data = [transfer_data_0, transfer_data_1]
        
        update_Tr_DB(transfer_data, CR, DB, self.COT)
        
        
        
        
        return d1
    
    def loan_to(self, owner, cost, squad, end_season = None, final_fee = None):
        (CR, DB, CO) = self.CC
        collection = self.collection
        d1 = update_owner(new_owner = owner, mode = None, squad = squad, loan = True, name = self.name, time_stamp = True, CR = CR, DB = DB, CO = CO)
        
        import datetime
        date = datetime.datetime.strptime(d1, '%Y/%m/%d')
        
        m, y = date.month, date.year
        if m > 7:
            y = y + 1
        
        loan_info = {
        'start_date': d1,
        'expire_date': str(y)+'/07/31', #all loans expire the successive July 31st
        'cost': cost,
        'redemption_info': end_season, #None, Option or Must
        'redemption_cost': final_fee, # None or >0
        }
        
        collection.update_one({'name': self.name}, {'$set':{'info.current_team.on_loan': True}}) 
        collection.update_one({'name': self.name}, {'$set':{'info.current_team.loan_info': loan_info}}) 
        
        transfer_data = [{
            'name': self.name,
            'operation': 'Prestito',
            'new_team': owner +', '+ squad,
            'loan_info': loan_info,
            'date': d1
                      }]
        
        
        update_Tr_DB(transfer_data, CR, DB, self.COT)
        
        return d1
    
    def end_loan(self, squad, redeemed = False):
        (CR, DB, CO) = self.CC
        collection = self.collection
        
        dic = self.info
        owner = dic['info']['contract']['owner']
        
        d1 = update_owner(new_owner = owner, mode = None, squad = squad, loan = True, name = self.name, time_stamp = True, CR = CR, DB = DB, CO = CO)
        
        collection.update_one({'name': self.name}, {'$set':{'info.current_team.on_loan': False}}) 
        collection.update_one({'name': self.name}, {'$set':{'info.current_team.loan_info': None}}) 
        
        transfer_data = [{
            'name': self.name,
            'operation': 'Fine Prestito',
            'new_team': owner +', '+ squad,
            'redeemed': redeemed,
            'date': d1
                      }]
        
        
        update_Tr_DB(transfer_data, CR, DB, self.COT)
        
        if redeemed:
            
            print('Registrare operazione di trasferimento!!!')
        
        return d1
        
        
    def move_up(self):
        (CR, DB, CO) = self.CC
        d1 = update_owner(new_owner = None, mode = None, squad = 'Main', move_up = True, name = self.name, time_stamp = True, CR = CR, DB = DB, CO = CO)
        
        transfer_data = [{
            'name': self.name,
            'operation': 'Promozione',
            'squad': 'Main',
            'date': d1
                      }]
        
        
        update_Tr_DB(transfer_data, CR, DB, self.COT)
        
        return d1
        
        

In [6]:
"""
This second layer is designed to prevent accidental changes in the database:
First all the players collection is duplicated to a temp one
then all the changes are made in the temp

NOTE: we need to add the equivalent for the transfer database, but we first need to design the db_update function
"""

class market_session:
    
    def __init__(self, ID, CR = cred['cred'], DB = 'Game', CO = 'Players', COT = 'Transfers'):
        #copy entire database into tempPlayers
        
        cluster = MongoClient(CR)
        db = cluster[DB]
        collection = db[CO]
        COt = 'tempPlayers'
        
        collectionT = db[COt]
        
        posts = list(collection.find({}))
        collectionT.insert_many(posts)
        
        self.ID = ID
        self.CC = (CR, DB, COt)
        self.CO = CO
        self.COTt = 'tempTransfers'
        
    def select_pl(self, name):
        (CR, DB, COt) = self.CC
        self.player = transfer(name, CR, DB, COt, self.COTt)
    
    def confirm_session(self):
        #here we span through both temp collections (players and transfers) and replace/add entries to the official database
        #then we erase the temp databases
        (CR, DB, COt) = self.CC
        CO = self.CO
        
        cluster = MongoClient(CR)
        db = cluster[DB]
        collection = db[CO]
        collectionT = db[COt]
        posts = list(collectionT.find({}))
        collection.delete_many({})
        collection.insert_many(posts)
        collectionT.delete_many({})
        
        coll_tr = db[COT]
        coll_trT = db[self.COTt]
        posts = list(coll_trT.find({}))
        coll_tr.delete_many({})
        coll_tr.insert_many(posts)
        coll_trT.delete_many({})
        
        
    def cancel_session(self):
        #erase temporary database: need to redefine market session
        (CR, DB, COt) = self.CC
        
        cluster = MongoClient(CR)
        db = cluster[DB]
        collectionT = db[COt]
        coll_trT = db[self.COTt]
        
        collectionT.delete_many({})
        coll_trT.delete_many({})

In [7]:
def player(name, session):
    #select one player and assign it to a variable 
    session.select_pl(name)
    return session.player #returns a class transfer object

In [8]:
#define market session with id number (not necessary)
#sess = market_session(1)

In [9]:
#create a transfer class object
musso = player('MUSSO', sess)

In [10]:
#other examples
ronaldo = player('RONALDO', sess)
lukaku = player('LUKAKU', sess)
ibra = player('IBRAHIMOVIC', sess)
immobile = player('IMMOBILE', sess)
orso = player('ORSOLINI', sess)

In [11]:
#check name
musso.name

'MUSSO'

In [12]:
#example draft
musso.draft_in('luca', squad = 'primavera')

Operation date: 


'2020/12/23'

In [13]:
#example auction
ronaldo.auction_in('enzo', squad = 'main', cost = 60)

Operation date: 


'2020/12/23'

In [14]:
#example buy cash
lukaku.cash_in(new_owner = 'nanni', squad = 'main', cost = 100)

Operation date: 


'2020/12/23'

In [15]:
#example of exchange operation. The point of view is of the receiving owner:
#the cost paid by whoever receives the exchange pl (ronaldo here)
#therefore, in this case, if IBRA's NEW owner pays, cost>0, viceversa, cost<0. squad is IBRA's future squad
ibra.exchange_with('MARTINEZ L.', cost = 4, squad = 'primavera', squad_exch_pl = 'main')

Operation date: 


'2020/12/23'

In [16]:
#example loan
immobile.loan_to(owner = 'luca', cost = 25, squad = 'main', end_season = 'Option', final_fee = 30)

Operation date: 


'2020/12/23'

In [17]:
#example end loan (note that in the same session this might cause problems, but it should never happen)
immobile.end_loan(squad = 'main', redeemed = True)

Operation date: 
Registrare operazione di trasferimento!!!


'2020/12/23'

In [18]:
#example buying after loan
immobile.cash_in(new_owner = 'luca', squad = 'main', cost = 30)

Operation date: 


'2020/12/23'

In [19]:
#example promotion
orso.move_up()

Operation date: 


'2020/12/23'

In [42]:
#CONFIRM: RUN ONLY IF TRANSFERS ARE FINAL

#sess.confirm_session()

#CANCEL: RUN TO DISCARD ALL THE TRANSFERS IN THE DEFINED SESSION

sess.cancel_session()

In [8]:
db['tempTransfers'].delete_many({})
db['tempPlayers'].delete_many({})

## Now I'll try to set this up using ipython widgets

In [9]:
import ipywidgets as widgets

### Session start/cancel/confirm
Run Start it at the beginning and Confirm at the end
If some error is made you can cancel the entire session by clicking Cancel

In [10]:
btn_start = widgets.Button(description='Start Session')
display(btn_start)
btn_cancel = widgets.Button(description='Cancel Session')
display(btn_cancel)
btn_confirm = widgets.Button(description='Confirm Session')
display(btn_confirm)
def sess_start_event(obj):
    global sess
    print('Session Started!', end="\r", flush=True)#.format(obj.description))
    sess = market_session(1)
def sess_cancel_event(obj):
    print('Session Canceled!', end="\r", flush=True)#.format(obj.description))
    sess.cancel_session()
def sess_confirm_event(obj):
    print('Session Confirmed!', end="\r", flush=True)#.format(obj.description))
    sess.cancel_session()
btn_start.on_click(sess_start_event)
btn_cancel.on_click(sess_cancel_event)
btn_confirm.on_click(sess_confirm_event)

Button(description='Start Session', style=ButtonStyle())

Button(description='Cancel Session', style=ButtonStyle())

Button(description='Confirm Session', style=ButtonStyle())

### Selection of the  player to be transfered

In [44]:
owners = ['Svincolati','luca','pietro','enzo','nanni','mario','emiliano','franky','musci8']


def owner_players(owner):
    if owner == 'Svincolati':
        results = db['tempPlayers'].find({'info.contract.owner': None})
        names = []
        for result in results:
            names.append(result['name'])
    else:
        results = db['tempPlayers'].find({'info.contract.owner': owner})
        names = []
        for result in results:
            names.append(result['name'])
    names.sort()
    return names

def confirm_player(Name):
    global pl
    pl = player(Name, sess)
    print('The player selected to be transferred is ' + Name)
    

def selection(pl_owner):
    global G_owner
    G_owner = pl_owner
    lista.options = owner_players(pl_owner)



scW = widgets.Dropdown(options=owners, description = 'Owner')

init = scW.value

lista = widgets.Dropdown(options=owner_players(init), description = 'Player')

j = widgets.interactive(confirm_player, Name=lista)

i = widgets.interactive(selection, pl_owner=scW)





# new_owner, squad, cost, exch_player, squad_exch_pl, end_season, final_fee


display(i)
display(j)

interactive(children=(Dropdown(description='Owner', options=('Svincolati', 'luca', 'pietro', 'enzo', 'nanni', …

interactive(children=(Dropdown(description='Player', options=('ACAMPORA', 'ADORANTE', 'AGOUME', 'AGUDELO', 'AK…

### Selection of parameters
Use only the ones that are relevant to the market operation

In [93]:
New_owner = widgets.Dropdown(options =    owners[1:], description = 'New Owner')

def new_owner_eventhandler(change):
    global new_owner
    new_owner = change.new
    print('After operation '+pl.name+' will be owned by ' + new_owner+'       ', end="\r", flush=True)

New_owner.observe(new_owner_eventhandler, names = 'value')

#display(New_owner)

squads = widgets.Dropdown(options = ['Main', 'Primavera'], description = 'New Squad')

def squad_eventhandler(change):
    global squad
    squad = change.new
    print('After operation '+pl.name+' will be in ' + new_owner+'\'s '+squad+' squad       ', end="\r", flush=True)

squads.observe(squad_eventhandler, names = 'value')

#display(squads)

cost_num = widgets.BoundedFloatText(min=-1000, max=1000, value=0, step=1, description = 'Cost')

def cost_num_eventhandler(change):
    global cost
    cost = change.new
    if cost >= 0:
        print(new_owner+' will pay ' + str(change.new)+' FM for the operation involving '+pl.name, end="\r", flush=True)
    else:
        print(new_owner+' will receive ' + str(- change.new)+' FM for the operation involving '+pl.name, end="\r", flush=True)
        
cost_num.observe(cost_num_eventhandler, names='value')


End_season = widgets.Dropdown(options = [None, 'Option', 'Must'], description = 'Redeem')

def End_season_eventhandler(change):
    global end_season
    end_season = change.new
    if end_season is None:
        print('If loan is chosen, no option is selected for end of the season             ', end="\r", flush=True)
    elif end_season is 'Option':
        print('If loan is chosen, '+pl.name+' CAN be redeemed at final fee value             ', end="\r", flush=True)
    else:
        print('If loan is chosen, '+pl.name+' MUST be redeemed at final fee value             ', end="\r", flush=True)

End_season.observe(End_season_eventhandler, names = 'value')



fee_num = widgets.BoundedFloatText(min=-1000, max=1000, value=0, step=1, description = 'Final fee')

def fee_num_eventhandler(change):
    global final_fee
    final_fee = change.new
    if cost >= 0:
        print(new_owner+' can/will pay ' + str(final_fee)+' to redeem '+pl.name+' at the end of the season            ', end="\r", flush=True)
        
fee_num.observe(fee_num_eventhandler, names='value')


#display(cost_num)

display(New_owner)
display(squads)
display(cost_num)

display(End_season)
display(fee_num)

Dropdown(description='New Owner', options=('luca', 'pietro', 'enzo', 'nanni', 'mario', 'emiliano', 'franky', '…

Dropdown(description='New Squad', options=('Main', 'Primavera'), value='Main')

BoundedFloatText(value=0.0, description='Cost', max=1000.0, min=-1000.0, step=1.0)

Dropdown(description='Redeem', options=(None, 'Option', 'Must'), value=None)

BoundedFloatText(value=0.0, description='Final fee', max=1000.0, min=-1000.0, step=1.0)

## To perform exchanges run cells below AFTER selecting parameters

In [84]:
Ex_players = widgets.Dropdown(options = owner_players(new_owner), description = 'Exch. Player')

def Ex_pl_eventhandler(change):
    global exch_pl
    exch_pl = change.new
    print('If exchange is chosen, '+pl.name+' will be exchanged with ' + exch_pl+'       ', end="\r", flush=True)
    
Ex_players.observe(Ex_pl_eventhandler, names = 'value')

#display(Ex_players)

Ex_squads = widgets.Dropdown(options = ['Main', 'Primavera'], description = 'Exch. Squad')

def squad_eventhandler(change):
    global squad_ex_pl
    squad_ex_pl = change.new
    print('After operation '+exch_pl+' will be in ' + G_owner+'\'s '+squad_ex_pl+' squad              ', end="\r", flush=True)

Ex_squads.observe(squad_eventhandler, names = 'value')

display(Ex_players)
display(Ex_squads)

Dropdown(description='Exch. Player', options=('BONAZZOLI', 'CALHANOGLU', 'CANDREVA', 'CETIN', 'CHIESA', "D'AMB…

Dropdown(description='Exch. Squad', options=('Main', 'Primavera'), value='Main')

### Check variables

In [14]:
#player to be transfered
pl.name

'ACAMPORA'

In [24]:
#new owner
new_owner

'luca'

In [82]:
#squad
squad

'Main'

In [91]:
# end season option
end_season

'Option'

In [92]:
# final fee
final_fee

10.0

In [88]:
#exchange player
exch_pl

'CETIN'

In [89]:
#cost of operation paid by new owner
cost

10.0

### From now on, I would not used widgets anymore, use scripting

In [ ]:
#example draft
pl.draft_in('luca', squad = 'primavera')

In [ ]:
btn_draft = widgets.Button(description='Draft Player')
display(btn_draft)
def draft_event(obj):
    print('Session Started!', end="\r", flush=True)#.format(obj.description))
    pl.draft_in(new_owner, squad = squad, cost = cost)

btn_draft.on_click(draft_event)

In [ ]:
btn_start = widgets.Button(description='Start Session')
display(btn_start)
btn_cancel = widgets.Button(description='Cancel Session')
display(btn_cancel)
btn_confirm = widgets.Button(description='Confirm Session')
display(btn_confirm)
def sess_start_event(obj):
    global sess
    print('Session Started!', end="\r", flush=True)#.format(obj.description))
    sess = market_session(1)
def sess_cancel_event(obj):
    print('Session Canceled!', end="\r", flush=True)#.format(obj.description))
    sess.cancel_session()
def sess_confirm_event(obj):
    print('Session Confirmed!', end="\r", flush=True)#.format(obj.description))
    sess.cancel_session()
btn_start.on_click(sess_start_event)
btn_cancel.on_click(sess_cancel_event)
btn_confirm.on_click(sess_confirm_event)